In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [3]:
import jieba

In [31]:
import re

第一步：分词

In [4]:
text = """我是一条天狗呀！
    我把月来吞了，
    我把日来吞了，
    我把一切的星球来吞了，
    我把全宇宙来吞了。
    我便是我了！"""
sentences = text.split()
sent_words = [list(jieba.cut(sent0)) for sent0 in sentences]
document = [" ".join(sent0) for sent0 in sent_words]
print(document)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\45563\AppData\Local\Temp\jieba.cache
Loading model cost 1.432 seconds.
Prefix dict has been built succesfully.


['我 是 一条 天狗 呀 ！', '我 把 月 来 吞 了 ，', '我 把 日来 吞 了 ，', '我 把 一切 的 星球 来 吞 了 ，', '我 把 全宇宙 来 吞 了 。', '我 便是 我 了 ！']


第二步：建模
- 理论上，现在得到的document的格式已经可以直接拿来训练了。让我们跑一下模型试试。

In [5]:
tfidf_model = TfidfVectorizer().fit(document)
print(tfidf_model.vocabulary_)
# {'一条': 1, '天狗': 4, '日来': 5, '一切': 0, '星球': 6, '全宇宙': 3, '便是': 2}
sparse_result = tfidf_model.transform(document)
print(sparse_result)
# (0, 4)	0.707106781187
# (0, 1)	0.707106781187
# (2, 5)	1.0
# (3, 6)	0.707106781187
# (3, 0)	0.707106781187
# (4, 3)	1.0
# (5, 2)	1.0

{'一条': 1, '天狗': 4, '日来': 5, '一切': 0, '星球': 6, '全宇宙': 3, '便是': 2}
  (0, 4)	0.7071067811865476
  (0, 1)	0.7071067811865476
  (2, 5)	1.0
  (3, 6)	0.7071067811865476
  (3, 0)	0.7071067811865476
  (4, 3)	1.0
  (5, 2)	1.0


导包：贝叶斯——GaussianNB、MultinomialNB、BernoulliNB

In [6]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

In [7]:
#引入数据
inputfile='./xinlang.csv'

In [10]:
df=pd.read_csv(inputfile,header=None)
df.head()

,0,1,2,3,4
0,1033036,宁德时代在互动平台表示，公司专注于新能源汽车动力电池系统、储能系统的研发、生产和销售，致力于...,2018-08-24 14:04:37,2018-08-24 14:04:49,"['公司', 'A股']"
1,1033037,刘昆表示，积极财政政策不是“大水漫灌”的强刺激，也不是要突破债务风险底限的安排，更不是要政府...,2018-08-24 14:07:30,2018-08-24 14:07:36,['宏观']
2,1033038,德国联邦金融监管局：银行正在放宽对企业的借贷标准。（德国经济周刊）,2018-08-24 14:07:55,2018-08-24 14:08:01,['行业']
3,1033039,交通运输部召开例行发布会，公布了《2017年全国收费公路统计公报》。交通部公路局副局长孙永红...,2018-08-24 14:10:36,2018-08-24 14:10:43,['宏观']
4,1033040,港元兑美元跌至7.8500，触及弱方兑换保证水平。,2018-08-24 14:11:22,2018-08-24 14:11:32,['市场']


In [12]:
news=df[1]
cls=df[4]
display(news,cls)

0       宁德时代在互动平台表示，公司专注于新能源汽车动力电池系统、储能系统的研发、生产和销售，致力于...
1       刘昆表示，积极财政政策不是“大水漫灌”的强刺激，也不是要突破债务风险底限的安排，更不是要政府...
2                       德国联邦金融监管局：银行正在放宽对企业的借贷标准。（德国经济周刊）
3       交通运输部召开例行发布会，公布了《2017年全国收费公路统计公报》。交通部公路局副局长孙永红...
4                               港元兑美元跌至7.8500，触及弱方兑换保证水平。
5       8月24日，在中国国际智能产业博览会期间，长安汽车举行2018智能化战略发布会，公布长安汽车...
6       调查发现，8月23日，北京新发地市场的西红柿价格约2-2.7元每斤，而22日仅需要2元。事实...
7        澳大利亚新总理莫里森：我相信一个“机会均等”的澳大利亚。就任后的先决任务就是处理旱灾地区的需要。
8       武汉东湖高新集团股份有限公司23日与重庆两江新区签署协议，双方将深度合作共同建设两江半导体产...
9       24日午后，猪肉股华统股份(002840)直线涨停。公司今日在互动平台澄清，浙江省温州市乐清...
10                东湖高新直线封板，消息面上，武汉东湖高新集团落子重庆，打造尖端产业"芯"高地。
11                                         韩国首尔综指数收涨0.5%。
12      交通运输部公路局副局长孙永红今天透露，公路建设资金超过八成都是依靠举借债务和社会资本的投资。...
13      北京市住建委日前会同市公安局、市工商局、市非紧急救助服务中心等部门，开通了12345打击“黑...
14      近期，一些不法分子打着“金融创新”“区块链”的旗号，通过发行所谓“虚拟货币”“虚拟资产”“数...
15      截止收盘，沪指报2729.43点，涨幅0.18%，成交额1027亿元，深成指报8484.74...
16                                 欧股开盘，欧洲Stoxx600开盘几无变动。
17      期市收盘，前

0             ['公司', 'A股']
1                   ['宏观']
2                   ['行业']
3                   ['宏观']
4                   ['市场']
5                   ['公司']
6                   ['其他']
7                   ['观点']
8                   ['行业']
9             ['公司', 'A股']
10            ['公司', 'A股']
11                  ['市场']
12                  ['其他']
13                  ['其他']
14                  ['焦点']
15            ['焦点', 'A股']
16                  ['市场']
17                  ['市场']
18                  ['公司']
19                  ['公司']
20            ['行业', '市场']
21            ['公司', 'A股']
22            ['公司', 'A股']
23                  ['公司']
24                  ['其他']
25            ['公司', 'A股']
26            ['宏观', '行业']
27                  ['观点']
28            ['公司', 'A股']
29            ['公司', 'A股']
               ...        
3904          ['宏观', '焦点']
3905                ['宏观']
3906          ['宏观', '焦点']
3907          ['宏观', '观点']
3908          ['宏观', '行业']
3909          ['公司', 'A股']
3

In [15]:
s='宁德时代在互动平台表示，公司专注于新能源汽车动力电池系统、储能系统的研发、生产和销售，致力于'
' '.join(jieba.cut(s))

'宁德 时代 在 互动 平台 表示 ， 公司 专注 于 新能源 汽车 动力电池 系统 、 储能 系统 的 研发 、 生产 和 销售 ， 致力于'

In [60]:
df[1]=df[1].map(lambda x:' '.join(jieba.cut(x)) )
df.head()

,0,1,2,3,4,5
0,1033036,宁德 时代 在 互动 平台 表示 ， 公司 专注 于 新能源 汽车 动力电池 系统 、 储能...,2018-08-24 14:04:37,2018-08-24 14:04:49,"['公司', 'A股']",公司 A股
1,1033037,刘昆 表示 ， 积极 财政政策 不是 “ 大水 漫灌 ” 的 强刺激 ， 也 不是 要 突破...,2018-08-24 14:07:30,2018-08-24 14:07:36,['宏观'],宏观
2,1033038,德国联邦 金融 监管局 ： 银行 正在 放宽 对 企业 的 借贷 标准 。 （ 德国 经济 ...,2018-08-24 14:07:55,2018-08-24 14:08:01,['行业'],行业
3,1033039,交通运输 部 召开 例行 发布会 ， 公布 了 《 2017 年 全国 收费公路 统计 公报...,2018-08-24 14:10:36,2018-08-24 14:10:43,['宏观'],宏观
4,1033040,港元 兑 美元 跌至 7.8500 ， 触及 弱方 兑换 保证 水平 。,2018-08-24 14:11:22,2018-08-24 14:11:32,['市场'],市场


In [57]:
d4=(((df[4].str.replace("[",'')).str.replace("]",'')).str.replace("', '",' ')).str.replace("'","")
d4

0          公司 A股
1             宏观
2             行业
3             宏观
4             市场
5             公司
6             其他
7             观点
8             行业
9          公司 A股
10         公司 A股
11            市场
12            其他
13            其他
14            焦点
15         焦点 A股
16            市场
17            市场
18            公司
19            公司
20         行业 市场
21         公司 A股
22         公司 A股
23            公司
24            其他
25         公司 A股
26         宏观 行业
27            观点
28         公司 A股
29         公司 A股
          ...   
3904       宏观 焦点
3905          宏观
3906       宏观 焦点
3907       宏观 观点
3908       宏观 行业
3909       公司 A股
3910       公司 A股
3911       公司 A股
3912       公司 A股
3913       公司 A股
3914    公司 焦点 A股
3915    公司 焦点 A股
3916       公司 A股
3917       公司 A股
3918          公司
3919    公司 焦点 A股
3920       宏观 焦点
3921       公司 A股
3922       公司 A股
3923       公司 A股
3924       公司 A股
3925          公司
3926          宏观
3927       公司 A股
3928       公司 A股
3929       宏观 焦点
3930       行业 观点
3931          

In [61]:
df[5]=d4
df.head()

,0,1,2,3,4,5
0,1033036,宁德 时代 在 互动 平台 表示 ， 公司 专注 于 新能源 汽车 动力电池 系统 、 储能...,2018-08-24 14:04:37,2018-08-24 14:04:49,"['公司', 'A股']",公司 A股
1,1033037,刘昆 表示 ， 积极 财政政策 不是 “ 大水 漫灌 ” 的 强刺激 ， 也 不是 要 突破...,2018-08-24 14:07:30,2018-08-24 14:07:36,['宏观'],宏观
2,1033038,德国联邦 金融 监管局 ： 银行 正在 放宽 对 企业 的 借贷 标准 。 （ 德国 经济 ...,2018-08-24 14:07:55,2018-08-24 14:08:01,['行业'],行业
3,1033039,交通运输 部 召开 例行 发布会 ， 公布 了 《 2017 年 全国 收费公路 统计 公报...,2018-08-24 14:10:36,2018-08-24 14:10:43,['宏观'],宏观
4,1033040,港元 兑 美元 跌至 7.8500 ， 触及 弱方 兑换 保证 水平 。,2018-08-24 14:11:22,2018-08-24 14:11:32,['市场'],市场


In [63]:
feature=df[1]
target=df[5]
display(feature,target)

0       宁德 时代 在 互动 平台 表示 ， 公司 专注 于 新能源 汽车 动力电池 系统 、 储能...
1       刘昆 表示 ， 积极 财政政策 不是 “ 大水 漫灌 ” 的 强刺激 ， 也 不是 要 突破...
2       德国联邦 金融 监管局 ： 银行 正在 放宽 对 企业 的 借贷 标准 。 （ 德国 经济 ...
3       交通运输 部 召开 例行 发布会 ， 公布 了 《 2017 年 全国 收费公路 统计 公报...
4                    港元 兑 美元 跌至 7.8500 ， 触及 弱方 兑换 保证 水平 。
5       8 月 24 日 ， 在 中国 国际 智能 产业 博览会 期间 ， 长安汽车 举行 2018...
6       调查 发现 ， 8 月 23 日 ， 北京 新 发地 市场 的 西红柿 价格 约 2 - 2...
7       澳大利亚 新 总理 莫里森 ： 我 相信 一个 “ 机会均等 ” 的 澳大利亚 。 就任 后...
8       武汉 东湖 高新 集团股份 有限公司 23 日 与 重庆 两江 新区 签署 协议 ， 双方 ...
9       24 日 午后 ， 猪肉 股华统 股份 ( 002840 ) 直线 涨停 。 公司 今日 在...
10      东湖 高新 直线 封板 ， 消息 面上 ， 武汉 东湖 高新 集团 落子 重庆 ， 打造 尖...
11                                   韩国 首尔 综 指数 收涨 0.5% 。
12      交通运输 部 公路局 副局长 孙永红 今天 透露 ， 公路 建设 资金 超过 八成 都 是 ...
13      北京市 住 建委 日前 会同 市 公安局 、 市工商局 、 市非 紧急 救助 服务中心 等 ...
14      近期 ， 一些 不法分子 打着 “ 金融 创新 ” “ 区块 链 ” 的 旗号 ， 通过 发...
15      截止 收盘 ， 沪 指报 2729.43 点 ， 涨幅 0.18% ， 成交额 1027 亿...
16                         欧股 开盘 ， 欧洲 Stoxx600 开盘 几无 变动 。
17      期 市 收盘

0          公司 A股
1             宏观
2             行业
3             宏观
4             市场
5             公司
6             其他
7             观点
8             行业
9          公司 A股
10         公司 A股
11            市场
12            其他
13            其他
14            焦点
15         焦点 A股
16            市场
17            市场
18            公司
19            公司
20         行业 市场
21         公司 A股
22         公司 A股
23            公司
24            其他
25         公司 A股
26         宏观 行业
27            观点
28         公司 A股
29         公司 A股
          ...   
3904       宏观 焦点
3905          宏观
3906       宏观 焦点
3907       宏观 观点
3908       宏观 行业
3909       公司 A股
3910       公司 A股
3911       公司 A股
3912       公司 A股
3913       公司 A股
3914    公司 焦点 A股
3915    公司 焦点 A股
3916       公司 A股
3917       公司 A股
3918          公司
3919    公司 焦点 A股
3920       宏观 焦点
3921       公司 A股
3922       公司 A股
3923       公司 A股
3924       公司 A股
3925          公司
3926          宏观
3927       公司 A股
3928       公司 A股
3929       宏观 焦点
3930       行业 观点
3931          

In [151]:
df[5][0]

'公司 A股'

In [64]:
df[1].size

3934

In [88]:
X_train,x_test,y_train,y_test=train_test_split(feature,target,test_size=0.2)

In [89]:
X_train.shape

(3147,)

In [90]:
y_train.shape

(3147,)

In [108]:
#将文本转换成词频
fea_m=TfidfVectorizer().fit(feature)

#训练的词频特征数次
tf_x_train=fea_m.transform(X_train)
#测试的词频特征数据
tf_x_test=fea_m.transform(x_test)


In [116]:
print(fea_m.vocabulary_)

{'宁德': 8636, '时代': 11848, '互动': 3884, '平台': 9520, '表示': 16802, '公司': 5218, '专注': 3096, '新能源': 11622, '汽车': 13092, '动力电池': 6123, '系统': 15517, '储能': 4964, '研发': 14819, '生产': 14232, '销售': 18707, '致力于': 16345, '全球': 5157, '应用': 9672, '提供': 11161, '一流': 2604, '解决方案': 16954, '暂未': 12055, '考虑': 16043, '进入': 18014, '领域': 19289, '此外': 12818, '再次': 5466, '重申': 18485, '特斯拉': 13981, '合作': 7118, '刘昆': 5812, '积极': 15053, '财政政策': 17348, '不是': 3017, '大水': 8272, '漫灌': 13725, '强刺激': 9948, '突破': 15172, '债务': 4868, '风险': 19338, '底限': 9689, '安排': 8679, '政府': 11391, '大包大揽': 8219, '目前': 14616, '而言': 16057, '更加': 12087, '增强': 7966, '政策': 11398, '前瞻性': 5994, '灵活性': 13792, '有效性': 12208, '结构调整': 15747, '方面': 11672, '发挥': 6855, '作用': 4627, '包括': 6191, '减税': 5596, '降费': 18950, '保持': 4750, '财政支出': 17347, '强度': 9957, '路透': 17677, '德国联邦': 10119, '金融': 18553, '监管局': 14582, '银行': 18681, '正在': 12784, '放宽': 11370, '企业': 4398, '借贷': 4857, '标准': 12572, '德国': 10117, '经济': 15689, '周刊': 7318, '交通运输': 4028, '召开': 6986, '例行': 4

In [115]:
display(tf_x_train.shape,X_train.shape,tf_x_test.shape,x_test.shape)

(3147, 19728)

(3147,)

(787, 19728)

(787,)

In [118]:
display(tf_x_train.shape,y_train.shape)

(3147, 19728)

(3147,)

In [120]:
mu=MultinomialNB()
ga=GaussianNB()
be=BernoulliNB()

In [121]:
mu.fit(tf_x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [122]:
mu.score(tf_x_test,y_test)

0.33290978398983484

In [126]:
ga.fit(tf_x_train.toarray(),y_train)

GaussianNB(priors=None)

In [128]:
ga.score(tf_x_test.toarray(),y_test)

0.3748411689961881

In [125]:
be.fit(tf_x_train,y_train)
be.score(tf_x_test,y_test)

0.2795425667090216

In [135]:
str1='刘昆表示，积极财政政策不是“大水漫灌”的强刺激，也不是要突破债务风险底限的安排'
str2=[' '.join(jieba.cut(str1))]
str2

['刘昆 表示 ， 积极 财政政策 不是 “ 大水 漫灌 ” 的 强刺激 ， 也 不是 要 突破 债务 风险 底限 的 安排']

In [137]:
test_str=fea_m.transform(str2)
test_str

<1x19728 sparse matrix of type '<class 'numpy.float64'>'
	with 13 stored elements in Compressed Sparse Row format>

In [138]:
mu.predict(test_str)

array(['宏观'], dtype='<U8')

In [145]:
ss='港元兑美元跌至7.8500，触及弱方兑换保证水平。'

In [147]:
ss='宁德时代在互动平台表示，公司专注于新能源汽车动力电池系统、储能系统的研发、生产和销售'
ss=[' '.join(jieba.cut(ss))]
ss

['宁德 时代 在 互动 平台 表示 ， 公司 专注 于 新能源 汽车 动力电池 系统 、 储能 系统 的 研发 、 生产 和 销售']

In [148]:
test_ss=fea_m.transform(ss)
test_ss

<1x19728 sparse matrix of type '<class 'numpy.float64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [149]:
mu.predict(test_ss)

array(['公司 A股'], dtype='<U8')

In [161]:
mu.predict(tf_x_test)

array(['公司', '公司', '公司 A股', '宏观', '公司 A股', '公司', '公司', '公司 A股', '公司 A股',
       '公司 A股', '公司 A股', '公司 A股', '公司 A股', '公司', '公司 A股', '公司', '公司 A股',
       '宏观', '宏观', '公司', '宏观', '宏观', '公司 A股', '市场', '市场', '公司 A股',
       '公司 A股', '公司 A股', '公司 A股', '公司 A股', '市场', '宏观', '公司', '公司 A股',
       '公司', '公司 A股', '公司 A股', '公司 A股', '公司 A股', '公司 A股', '市场', '公司',
       '公司', '公司 A股', '公司 A股', '公司', '公司', '公司', '公司 A股', '市场', '公司 A股',
       '公司', '宏观', '公司 A股', '公司', '宏观', '公司 A股', '公司 A股', '宏观', '公司 A股',
       '公司 A股', '公司 A股', '公司 A股', '市场', '宏观', '市场', '宏观', '公司 A股', '市场',
       '公司 A股', '公司 A股', '公司 A股', '公司 A股', '公司 A股', '公司', '公司 A股',
       '公司 A股', '宏观', '宏观', '宏观', '市场', '市场', '市场', '公司', '公司 A股',
       '公司 A股', '公司 A股', '公司 A股', '公司', '公司 A股', '公司 A股', '公司 A股', '公司',
       '公司 A股', '公司 A股', '市场', '公司 A股', '公司', '公司', '公司 A股', '公司 A股',
       '公司 A股', '市场', '公司 A股', '公司 A股', '公司', '公司 A股', '公司 A股', '公司',
       '市场', '公司 A股', '宏观', '公司 A股', '公司 A股', '公司 A股', '公司 A股', '公司 A股',
       '

In [167]:
(mu.predict(tf_x_test)==y_test).sum()/y_test.size

0.33290978398983484

In [156]:
demo=pd.read_csv('./sina2.csv',header=None)
demo.head(2)

,0,1,2,3,4
0,1043805,海得控制公告，公司5月29日曾披露，董事赵大砥因个人资金需求，计划披露之日起15个交易日后...,2018-09-13 15:55:25,2018-09-13 15:55:38,"['公司', 'A股']"
1,1043806,恒瑞医药公告，国家食药监局药品审评中心近日发布《拟纳入优先审评程序药品注册申请的公示（第三...,2018-09-13 15:57:31,2018-09-13 15:57:40,"['公司', 'A股']"


In [158]:
display(demo[1],demo[4])

0       海得控制公告，公司5月29日曾披露，董事赵大砥因个人资金需求，计划披露之日起15个交易日后...
1       恒瑞医药公告，国家食药监局药品审评中心近日发布《拟纳入优先审评程序药品注册申请的公示（第三...
2      在近日召开的“2018年度亚金协·东京主论坛”上，中国银行保险监督管理委员会副主席黄洪表示，...
3       乾景园林公告，公司成为北京市海淀区东升科技园二期周边绿地建设工程二标段（施工）项目的第一中...
4                      国际能源署（IEA）警告称，随着伊朗、委内瑞拉减产，油价可能上涨。
5      IEA月报：2019年全球原油需求增速预期为140万桶/日，此前为149万桶/日。 预计20...
6      港股尾盘发力，恒指涨2.5%收复27000点关口，势创两年半最大单日涨幅。连续四日回购，腾讯...
7       IEA月报：8月伊朗石油产量环比减少15万桶/日至363万桶/日，为2016年7月以来最低水平。
8                              LME锌库存减少2.6%，为2017以来最大降幅。
9      太极实业公告，近日，控股子公司十一科技与青岛芯恩签订了《EPC总承包工程合同》，合同金额为1...
10      9月13日，泰豪科技在互动平台表示，公司持有30%股权的江西国科未出现在最近一期证监会公布...
11     “市场对于今年我国的玉米产量存在分歧，一些人认为同比会下降，但根据我们的调查，今年我国玉米产...
12                       恒指收涨2.54%收复27000点关口，创两年半最大单日涨幅。
13                            WTI原油短线小幅走低，目前跌1%报69.67美元。
14     浪潮信息近日在机构调研时表示，目前，公司出口美国业务占公司总营收占比不高，公司核心部件采取全...
15                                     互联互通南向6.3亿元净流出港股。
16     迪生力9月13日晚间公告，拟在香港新设立全资子公司广东迪生力汽配（香港）有限公司。香港子公司...
17     顾家家居9月13日晚间公告可转债发行网上中签结果

0            ['公司', 'A股']
1            ['公司', 'A股']
2      ['行业', '数据', 'A股']
3            ['公司', 'A股']
4            ['市场', '焦点']
5            ['数据', '焦点']
6                  ['市场']
7            ['数据', '市场']
8                  ['市场']
9            ['公司', 'A股']
10           ['公司', 'A股']
11                 ['宏观']
12           ['宏观', '焦点']
13                 ['市场']
14           ['公司', 'A股']
15                 ['市场']
16           ['公司', 'A股']
17           ['公司', 'A股']
18                 ['公司']
19                 ['公司']
20                 ['公司']
21           ['公司', 'A股']
22           ['公司', 'A股']
23                 ['市场']
24                 ['宏观']
25           ['公司', 'A股']
26           ['公司', 'A股']
27     ['公司', '焦点', 'A股']
28                 ['宏观']
29           ['公司', 'A股']
              ...        
792                ['其他']
793                ['公司']
794          ['行业', '公司']
795                ['其他']
796                ['公司']
797          ['行业', '观点']
798                ['其他']
799         

In [159]:
demo.size

4110

In [169]:
demo[5]=(((demo[4].str.replace("[",'')).str.replace("]",'')).str.replace("', '",' ')).str.replace("'","")
demo.head(2)

,0,1,2,3,4,5
0,1043805,海得控制公告，公司5月29日曾披露，董事赵大砥因个人资金需求，计划披露之日起15个交易日后...,2018-09-13 15:55:25,2018-09-13 15:55:38,"['公司', 'A股']",公司 A股
1,1043806,恒瑞医药公告，国家食药监局药品审评中心近日发布《拟纳入优先审评程序药品注册申请的公示（第三...,2018-09-13 15:57:31,2018-09-13 15:57:40,"['公司', 'A股']",公司 A股


In [171]:
demo[1]=demo[1].map(lambda x:' '.join(jieba.cut(x)))
demo.head(2)

,0,1,2,3,4,5
0,1043805,海得 控制 公告 ， 公司 5 月 29 日 曾 披露 ， 董事 赵大砥因 个人 资金 ...,2018-09-13 15:55:25,2018-09-13 15:55:38,"['公司', 'A股']",公司 A股
1,1043806,恒瑞医药 公告 ， 国家 食 药监局 药品 审评 中心 近日 发布 《 拟 纳入 优先 ...,2018-09-13 15:57:31,2018-09-13 15:57:40,"['公司', 'A股']",公司 A股


In [172]:
f=demo[1]
t=demo[5]

In [174]:
f_test=fea_m.transform(f)
f_test

<822x19728 sparse matrix of type '<class 'numpy.float64'>'
	with 21931 stored elements in Compressed Sparse Row format>

In [176]:
f_y_=mu.predict(f_test)
f_y_

array(['公司 A股', '公司 A股', '公司 A股', '公司 A股', '市场', '市场', '市场', '公司 A股',
       '市场', '公司 A股', '公司 A股', '公司 A股', '市场', '市场', '公司 A股', '市场',
       '公司 A股', '公司 A股', '公司', '公司', '公司', '公司 A股', '公司 A股', '市场',
       '公司 A股', '公司 A股', '公司 A股', '公司 A股', '公司 A股', '公司 A股', '市场',
       '公司 A股', '公司 A股', '市场', '宏观', '公司 A股', '公司 A股', '宏观', '公司',
       '公司 A股', '公司 A股', '公司 A股', '公司 A股', '公司 A股', '宏观', '宏观', '公司 A股',
       '公司 A股', '公司 A股', '公司 A股', '公司 A股', '公司 A股', '公司 A股', '公司 A股',
       '市场', '宏观', '市场', '公司 A股', '公司 A股', '公司', '宏观', '公司 A股', '公司 A股',
       '市场', '公司 A股', '公司 A股', '公司', '公司', '市场', '公司 A股', '公司 A股',
       '公司 A股', '公司 A股', '公司 A股', '宏观', '公司', '公司 A股', '公司 A股', '宏观',
       '公司 A股', '公司 A股', '公司', '公司 A股', '公司 A股', '公司 A股', '宏观', '宏观',
       '公司', '市场', '公司 A股', '宏观', '公司 A股', '宏观', '公司', '公司 A股', '公司 A股',
       '公司 A股', '公司', '公司', '公司 A股', '公司 A股', '公司 A股', '市场', '公司 A股',
       '市场', '宏观', '公司 A股', '公司 A股', '公司 A股', '公司 A股', '公司 A股', '公司 A股',
       '公司 A股', '公司 A股'

In [224]:
mu.score(f_test,t)

0.38321167883211676

In [227]:
(f_y_==t).sum()/t.size

0.38321167883211676

In [229]:
# 新闻、测试分类、正确分类
news_p=pd.concat([DataFrame(demo[1]),DataFrame(f_y_),DataFrame(t)],axis=1)
news_p.columns=['news','predict','true']
news_p

,news,predict,true
0,海得 控制 公告 ， 公司 5 月 29 日 曾 披露 ， 董事 赵大砥因 个人 资金 ...,公司 A股,公司 A股
1,恒瑞医药 公告 ， 国家 食 药监局 药品 审评 中心 近日 发布 《 拟 纳入 优先 ...,公司 A股,公司 A股
2,在 近日 召开 的 “ 2018 年度 亚金 协 · 东京 主 论坛 ” 上 ， 中国银行 ...,公司 A股,行业 数据 A股
3,乾景 园林 公告 ， 公司 成为 北京市 海淀区 东升 科技园 二期 周边 绿地 建设工...,公司 A股,公司 A股
4,国际 能源 署 （ IEA ） 警告 称 ， 随着 伊朗 、 委内瑞拉 减产 ， 油价 可能...,市场,市场 焦点
5,IEA 月 报 ： 2019 年 全球 原油 需求 增速 预期 为 140 万桶 / 日 ，...,市场,数据 焦点
6,港股 尾盘 发力 ， 恒指 涨 2.5% 收复 27000 点 关口 ， 势创 两年 半 最...,市场,市场
7,IEA 月 报 ： 8 月 伊朗 石油 产量 环比 减少 15 万桶 / 日至 363 万桶...,公司 A股,数据 市场
8,LME 锌 库存 减少 2.6% ， 为 2017 以来 最大 降幅 。,市场,市场
9,太极 实业 公告 ， 近日 ， 控股 子公司 十一 科技 与 青岛 芯恩 签订 了 《 EP...,公司 A股,公司 A股
